In [4]:
import os
import numpy as np
from pandas import DataFrame
from pandas.io.parsers import read_csv
from datetime import datetime
%matplotlib inline 

from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from skimage import data, img_as_float
from skimage import exposure
import theano

from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet, BatchIterator

import warnings
warnings.filterwarnings("ignore")

ImportError: No module named lasagne

In [3]:
# Other Functions or Classes
# Function to load train/test csv files 
def load(test=False, cols=None):
    """Loads data from FTEST if *test* is True, otherwise from FTRAIN.
    Pass a list of *cols* if you're only interested in a subset of the
    target columns.
    """
    fname = FTEST if test else FTRAIN
    df = read_csv(os.path.expanduser(fname))  # load pandas dataframe

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    if cols:  # get a subset of columns
        df = df[list(cols) + ['Image']]

    print(df.count())  # prints the number of values for each column
    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['Image'].values) / 255.  # scale pixel values to [0, 1]
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[:-1]].values
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        y = y.astype(np.float32)
    else:
        y = None

    return X, y

# Class to display list as HTML Table in Jupyter Notebook
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)
    
def plot_sample(x, y, axis):
    img = x.reshape(96, 96)
    axis.imshow(img, cmap='gray')
    axis.scatter(y[0::2] * 48 + 48, y[1::2] * 48 + 48, marker='x', s=30, c='blue')

In [ ]:
FTRAIN = '/data/facial_keypoints_detection/training.csv'
FTEST = '/data/facial_keypoints_detection/test.csv'

# Load Test data
Test_X, _ = load(test=True)

# Load Train data
X, y = load()
print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(
    X.shape, X.min(), X.max()))
print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(
    y.shape, y.min(), y.max()))

In [ ]:
# a list of hiddern unit number
hidden_num_units = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# Define a listtable to hold statsitical outcomes of NNs with various hiddern unit numbers 
performance_table_1 = ListTable()
performance_table_1.append(['Num_hidden_layer', 'Train_loss', 'Validation_loss', 'Duration_time(s)'])

fig = plt.figure(figsize=(20,8))

for index,num in enumerate(hidden_num_units):
    net1 = NeuralNet(layers=[('input', layers.InputLayer), # three layers
                             ('hidden', layers.DenseLayer),
                             ('output', layers.DenseLayer),],
                     input_shape=(None, 9216),  # 96x96 input pixels per batch
                     #hidden_num_units=200,  # number of units in hidden layer
                     output_nonlinearity=None,  # output layer uses identity function
                     output_num_units=30,  # 30 target values
                     
                     update=nesterov_momentum, # optimization parameters
                     update_learning_rate=0.01,
                     update_momentum=0.9,
                     
                     regression=True,  # flag to indicate we're dealing with regression problem
                     max_epochs=1000,  # we want to train this many epochs
                     verbose=0,)
    net1.hidden_num_units = num
    net1.fit(X, y)
    
    train_loss = np.array([i["train_loss"] for i in net1.train_history_])
    valid_loss = np.array([i["valid_loss"] for i in net1.train_history_])
    print net1.hidden_num_units 
    print "Train, valid errors and computational time of last epcho: %0.4f, %0.4f, %0.4f" %(net1.train_history_[-1]["train_loss"], net1.train_history_[-1]["valid_loss"], net1.train_history_[-1]["dur"])
    performance_table_1.append([net1.hidden_num_units,
                                round(net1.train_history_[-1]["train_loss"],5),
                                round(net1.train_history_[-1]["valid_loss"],5),
                                round(net1.train_history_[-1]["dur"],5)] )
    
    subfig = fig.add_subplot(2, 5, index+1, xticks=[], yticks=[])
    plt.plot(train_loss, linewidth=3, color="blue", linestyle="dashed", label="train")
    plt.plot(valid_loss, linewidth=3, color="blue", linestyle="solid", label="valid")
    plt.grid()
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.ylim(5*1e-4, 1e-2)
    plt.xlim(0,1000)
    plt.yscale("log")
    subfig.set_title('Num_Hidden_Layer: ' + str(num))

plt.show()
performance_table_1  

In [ ]:
net1 = NeuralNet(layers=[('input', layers.InputLayer), # three layers
                         ('hidden', layers.DenseLayer),
                         ('output', layers.DenseLayer),],
                 input_shape=(None, 9216),  # 96x96 input pixels per batch
                 hidden_num_units=500,  # number of units in hidden layer
                 output_nonlinearity=None,  # output layer uses identity function
                 output_num_units=30,  # 30 target values
                 
                 update=nesterov_momentum, # optimization parameters
                 update_learning_rate=0.01,
                 update_momentum=0.9,
                 
                 regression=True,  # flag to indicate we're dealing with regression problem
                 max_epochs=1000,  # we want to train this many epochs
                 verbose=0,)

net1.fit(X, y)
print "Train, valid errors and computational time of last epcho: %0.5f, %0.5f, %0.4f" %(net1.train_history_[-1]["train_loss"], net1.train_history_[-1]["valid_loss"], net1.train_history_[-1]["dur"])

Test_y_pred = net1.predict(Test_X)

fig = plt.figure(figsize=(12, 16))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(40):
    ax = fig.add_subplot(8, 5, i + 1, xticks=[], yticks=[])
    plot_sample(Test_X[i], Test_y_pred[i], ax)

plt.show()

In [ ]:
print Test_X.shape

In [ ]:


X_stretch = np.empty_like(X)
X_eq = np.empty_like(X)
X_adapteq = np.empty_like(X)


for i in range(X.shape[0]):
    img = X[i,]
    p_lowbound, p_upperbound = np.percentile(img, (1, 99))
    X_stretch[i,] = exposure.rescale_intensity(img, in_range=(p_lowbound,p_upperbound ))
    
for i in range(X.shape[0]):
    img = X[i,]
    X_eq[i,] =  exposure.equalize_hist(img,nbins=255)

for i in range(X.shape[0]):
    img = X[i,].reshape(96,96)
    X_adapteq[i,] =  exposure.equalize_adapthist(img, clip_limit=0.03).flatten()
    

In [ ]:
print X_adapteq.shape
print X_adapteq[0]

In [ ]:
net1_stretch = NeuralNet(layers=[('input', layers.InputLayer), # three layers
                         ('hidden', layers.DenseLayer),
                         ('output', layers.DenseLayer),],
                 input_shape=(None, 9216),  # 96x96 input pixels per batch
                 hidden_num_units=500,  # number of units in hidden layer
                 output_nonlinearity=None,  # output layer uses identity function
                 output_num_units=30,  # 30 target values
                 
                 update=nesterov_momentum, # optimization parameters
                 update_learning_rate=0.01,
                 update_momentum=0.9,
                 
                 regression=True,  # flag to indicate we're dealing with regression problem
                 max_epochs=1000,  # we want to train this many epochs
                 verbose=0,)

net1_stretch.fit(X_stretch, y)
print "Train, valid errors and computational time of last epcho: %0.5f, %0.5f, %0.4f" %(net1_stretch.train_history_[-1]["train_loss"], net1_stretch.train_history_[-1]["valid_loss"], net1_stretch.train_history_[-1]["dur"])


net1_eq = NeuralNet(layers=[('input', layers.InputLayer), # three layers
                         ('hidden', layers.DenseLayer),
                         ('output', layers.DenseLayer),],
                 input_shape=(None, 9216),  # 96x96 input pixels per batch
                 hidden_num_units=500,  # number of units in hidden layer
                 output_nonlinearity=None,  # output layer uses identity function
                 output_num_units=30,  # 30 target values
                 
                 update=nesterov_momentum, # optimization parameters
                 update_learning_rate=0.01,
                 update_momentum=0.9,
                 
                 regression=True,  # flag to indicate we're dealing with regression problem
                 max_epochs=1000,  # we want to train this many epochs
                 verbose=0,)
net1_eq.fit(X_eq, y)
print "Train, valid errors and computational time of last epcho: %0.5f, %0.5f, %0.4f" %(net1_eq.train_history_[-1]["train_loss"], net1_eq.train_history_[-1]["valid_loss"], net1_eq.train_history_[-1]["dur"])

'''
net1_adapteq = NeuralNet(layers=[('input', layers.InputLayer), # three layers
                         ('hidden', layers.DenseLayer),
                         ('output', layers.DenseLayer),],
                 input_shape=(None, 9216),  # 96x96 input pixels per batch
                 hidden_num_units=500,  # number of units in hidden layer
                 output_nonlinearity=None,  # output layer uses identity function
                 output_num_units=30,  # 30 target values
                 
                 update=nesterov_momentum, # optimization parameters
                 update_learning_rate=0.01,
                 update_momentum=0.9,
                 
                 regression=True,  # flag to indicate we're dealing with regression problem
                 max_epochs=1000,  # we want to train this many epochs
                 verbose=0,)
net1_adapteq.fit(X_adapteq, y)
print "Train, valid errors and computational time of last epcho: %0.5f, %0.5f, %0.4f" %(net1_adapteq.train_history_[-1]["train_loss"], net1_adapteq.train_history_[-1]["valid_loss"], net1_adapteq.train_history_[-1]["dur"])
'''


In [ ]:
# Extract training and validation loss for various models
net1_train_loss = np.array([i["train_loss"] for i in net1.train_history_])
net1_valid_loss = np.array([i["valid_loss"] for i in net1.train_history_])
net1_str_train_loss = np.array([i["train_loss"] for i in net1_stretch.train_history_])
net1_str_valid_loss = np.array([i["valid_loss"] for i in net1_stretch.train_history_])
net1_eq_train_loss = np.array([i["train_loss"] for i in net1_eq.train_history_])
net1_eq_valid_loss = np.array([i["valid_loss"] for i in net1_eq.train_history_])
net1_adapteq_train_loss = np.array([i["train_loss"] for i in net1_adapteq.train_history_])
net1_adapteq_valid_loss = np.array([i["valid_loss"] for i in net1_adapteq.train_history_])

# Plot for performance comparision between differnt preprocesses
plt.figure(figsize=(10, 8))
plt.plot(net1_train_loss, linewidth=3,  color="green", linestyle="dashed", label="Train without preprocess")
plt.plot(net1_valid_loss, linewidth=3,  color="green", linestyle="solid", label="Valid without preprocess")
plt.plot(net1_str_train_loss, linewidth=3,  color="red", linestyle="dashed", label="Train after Contrast Stretching")
plt.plot(net1_str_valid_loss, linewidth=3,  color="red", linestyle="solid", label="Valid after Contrast Stretching")
plt.plot(net1_eq_train_loss, linewidth=3,  color="yellow", linestyle="dashed", label="Train after Histogram Equalization")
plt.plot(net1_eq_valid_loss, linewidth=3,  color="yellow", linestyle="solid", label="Valid after Histogram Equalization")

plt.grid()
plt.legend()
plt.xlabel("epoch")
plt.ylabel("loss")
plt.ylim(8*1e-4, 1e-2)
plt.yscale("log")

plt.show()

# Define a listtable to hold statsitical outcomes of NNs with various preprocessing steps 
performance_table_2 = ListTable()
performance_table_2.append(['Preprocess', 'Train_loss', 'Validation_loss', 'Duration_time(s)'])
performance_table_2.append(['Without Preprocessing',
                            round(net1.train_history_[-1]["train_loss"],5),
                            round(net1.train_history_[-1]["valid_loss"],5),
                            round(net1.train_history_[-1]["dur"],5)] )
performance_table_2.append(['Contrast stretching',
                            round(net1_stretch.train_history_[-1]["train_loss"],5),
                            round(net1_stretch.train_history_[-1]["valid_loss"],5),
                            round(net1_stretch.train_history_[-1]["dur"],5)] )    
performance_table_2.append(['Histogram equalization',
                            round(net1_eq.train_history_[-1]["train_loss"],5),
                            round(net1_eq.train_history_[-1]["valid_loss"],5),
                            round(net1_eq.train_history_[-1]["dur"],5)] )   
performance_table_2  




In [ ]:
from scipy import fftpack

def idct2(xIn2Dim):
    return fftpack.idct(fftpack.idct(xIn2Dim, norm = 'ortho').T, norm = 'ortho')

def dct2(xIn, xDim = 96, yDim = 96):
    #dctback = fftpack.idct(dct)

    return fftpack.dct(fftpack.dct(xIn.reshape(xDim,yDim), norm = 'ortho').T, norm = 'ortho')

def primHPF(xIn2Dim, level = 5):
    shape0 = xIn2Dim.shape[0]
    shape1 = xIn2Dim.shape[1]
    
    for i in range(shape0 - level, shape0):
        for j in range(shape1 - level, shape1):
            xIn2Dim[i,j] = 0
    return xIn2Dim

hpfX = np.empty_like(X)
for i in range(X.shape[0]):
    hpfX[i] = idct2(primHPF(dct2(X[i]))).reshape(96 * 96)
    
net1_hpf = NeuralNet(layers=[('input', layers.InputLayer), # three layers
                         ('hidden', layers.DenseLayer),
                         ('output', layers.DenseLayer),],
                 input_shape=(None, 9216),  # 96x96 input pixels per batch
                 hidden_num_units=500,  # number of units in hidden layer
                 output_nonlinearity=None,  # output layer uses identity function
                 output_num_units=30,  # 30 target values
                 
                 update=nesterov_momentum, # optimization parameters
                 update_learning_rate=0.01,
                 update_momentum=0.9,
                 
                 regression=True,  # flag to indicate we're dealing with regression problem
                 max_epochs=1000,  # we want to train this many epochs
                 verbose=0,)

net1_hpf.fit(hpfX, y)
print "Train, valid errors and computational time of last epcho: %0.5f, %0.5f, %0.4f" %(net1_hpf.train_history_[-1]["train_loss"], net1_hpf.train_history_[-1]["valid_loss"], net1_hpf.train_history_[-1]["dur"])


In [ ]:
from nolearn.lasagne import BatchIterator

# randomly flip half of the images in each batch
class FlipBatchIterator(BatchIterator):
    flip_indices = [
        (0, 2), (1, 3),
        (4, 8), (5, 9), (6, 10), (7, 11),
        (12, 16), (13, 17), (14, 18), (15, 19),
        (22, 24), (23, 25),
        ]

    def transform(self, Xb, yb):
        Xb, yb = super(FlipBatchIterator, self).transform(Xb, yb)

        # Flip half of the images in this batch at random:
        bs = Xb.shape[0]
        indices = np.random.choice(bs, bs / 2, replace=False)
        for i in range(0, np.int64(bs/2)):
            X_trans = Xb[indices[i]].reshape(96, 96)
            X_trans = X_trans[:, ::-1]
            Xb[indices[i]] = X_trans.reshape(1, 9216)

        if yb is not None:
            # Horizontal flip of all x coordinates:
            yb[indices, ::2] = yb[indices, ::2] * -1

            # Swap places, e.g. left_eye_center_x -> right_eye_center_x
            for a, b in self.flip_indices:
                yb[indices, a], yb[indices, b] = (
                    yb[indices, b], yb[indices, a])

        return Xb, yb


In [ ]:

net1_augmentation = NeuralNet(layers=[('input', layers.InputLayer), # three layers
                                      ('hidden', layers.DenseLayer),
                                      ('output', layers.DenseLayer),],
                              # layer parameters:
                              input_shape=(None, 9216),  # 96x96 input pixels per batch
                              hidden_num_units=500,  # number of units in hidden layer
                              output_nonlinearity=None,  # output layer uses identity function
                              output_num_units=30,  # 30 target values
                              
                              # optimization parameters:
                              update=nesterov_momentum,
                              update_learning_rate=0.01,
                              update_momentum=0.9,
                 
                              regression=True,  # flag to indicate we're dealing with regression problem
                              batch_iterator_train=FlipBatchIterator(batch_size=128),
                              max_epochs=1000,  # we want to train this many epocho
                              verbose=1,)

net1_augmentation.fit(X, y)

In [ ]:
net1_eq_augmentation = NeuralNet(layers=[('input', layers.InputLayer), # three layers
                                      ('hidden', layers.DenseLayer),
                                      ('output', layers.DenseLayer),],
                              # layer parameters:
                              input_shape=(None, 9216),  # 96x96 input pixels per batch
                              hidden_num_units=500,  # number of units in hidden layer
                              output_nonlinearity=None,  # output layer uses identity function
                              output_num_units=30,  # 30 target values
                              
                              # optimization parameters:
                              update=nesterov_momentum,
                              update_learning_rate=0.01,
                              update_momentum=0.9,
                 
                              regression=True,  # flag to indicate we're dealing with regression problem
                              batch_iterator_train=FlipBatchIterator(batch_size=128),
                              max_epochs=1000,  # we want to train this many epocho
                              verbose=1,)

net1_eq_augmentation.fit(X_eq, y)

In [ ]:
print "Train, valid errors and computational time of last epcho: %0.5f, %0.5f, %0.4f" %(net1_augmentation.train_history_[-1]["train_loss"], net1_augmentation.train_history_[-1]["valid_loss"], net1_augmentation.train_history_[-1]["dur"])
print "Train, valid errors and computational time of last epcho: %0.5f, %0.5f, %0.4f" %(net1_eq_augmentation.train_history_[-1]["train_loss"], net1_eq_augmentation.train_history_[-1]["valid_loss"], net1_augmentation.train_history_[-1]["dur"])


# Extract training and validation loss for various models
net1_train_loss = np.array([i["train_loss"] for i in net1.train_history_])
net1_valid_loss = np.array([i["valid_loss"] for i in net1.train_history_])
net1_eq_train_loss = np.array([i["train_loss"] for i in net1_eq.train_history_])
net1_eq_valid_loss = np.array([i["valid_loss"] for i in net1_eq.train_history_])
net1_augmentation_train_loss = np.array([i["train_loss"] for i in net1_augmentation.train_history_])
net1_augmentation_valid_loss = np.array([i["valid_loss"] for i in net1_augmentation.train_history_])
net1_eq_augmentation_train_loss = np.array([i["train_loss"] for i in net1_eq_augmentation.train_history_])
net1_eq_augmentation_valid_loss = np.array([i["valid_loss"] for i in net1_eq_augmentation.train_history_])

# Plot for performance comparision between differnt preprocesses
plt.figure(figsize=(10, 8))
plt.plot(net1_train_loss, linewidth=3,  color="green", linestyle="dashed", label="Train without preprocess")
plt.plot(net1_valid_loss, linewidth=3,  color="green", linestyle="solid", label="Valid without preprocess")
plt.plot(net1_eq_train_loss, linewidth=3,  color="yellow", linestyle="dashed", label="Train after Histogram Equalization")
plt.plot(net1_eq_valid_loss, linewidth=3,  color="yellow", linestyle="solid", label="Valid after Histogram Equalization")
plt.plot(net1_augmentation_train_loss, linewidth=3,  color="purple", linestyle="dashed", label="Train after Data Argumentation")
plt.plot(net1_augmentation_valid_loss, linewidth=3,  color="purple", linestyle="solid", label="Valid after Data Argumentation")
plt.plot(net1_eq_augmentation_train_loss, linewidth=3,  color="black", linestyle="dashed", label="Train after Data Argumentation & Histogram Equalization")
plt.plot(net1_eq_augmentation_valid_loss, linewidth=3,  color="black", linestyle="solid", label="Valid after Data Argumentation & Histogram Equalization")


plt.grid()
plt.legend()
plt.xlabel("epoch")
plt.ylabel("loss")
plt.ylim(8*1e-4, 1e-2)
plt.yscale("log")

plt.show()



In [ ]:
# a list of dropout rate
dropout_rate = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45]

# Define a listtable to hold statsitical outcomes of NNs with various hiddern unit numbers 
performance_table_3 = ListTable()
performance_table_3.append(['Dropout_rate', 'Train_loss', 'Validation_loss', 'Duration_time(s)'])

fig = plt.figure(figsize=(20,8))

for index,num in enumerate(dropout_rate):
    net1_dropout = NeuralNet(layers=[('input', layers.InputLayer), # three layers
                                     ('hidden', layers.DenseLayer),
                                     ('dropout1', layers.DropoutLayer),
                                     ('output', layers.DenseLayer),],
                             
                             input_shape=(None, 9216),  # 96x96 input pixels per batch
                             hidden_num_units=500,  # number of units in hidden layer
                             output_nonlinearity=None,  # output layer uses identity function
                             output_num_units=30,  # 30 target values
                             
                             update=nesterov_momentum, # optimization parameters
                             update_learning_rate=0.01,
                             update_momentum=0.9,
                             
                             regression=True,  # flag to indicate we're dealing with regression problem
                             max_epochs=1000,  # we want to train this many epochs
                             verbose=0,)
    net1_dropout.dropout1_p = num
    net1_dropout.fit(X, y)
    
    train_loss = np.array([i["train_loss"] for i in net1_dropout.train_history_])
    valid_loss = np.array([i["valid_loss"] for i in net1_dropout.train_history_])
    print "Dropout Rate is: %.2f" %net1_dropout.dropout1_p 
    print "Train, valid errors and computational time of last epcho: %0.4f, %0.4f, %0.4f" %(net1_dropout.train_history_[-1]["train_loss"], net1_dropout.train_history_[-1]["valid_loss"], net1_dropout.train_history_[-1]["dur"])
    performance_table_3.append([net1_dropout.dropout1_p,
                                round(net1_dropout.train_history_[-1]["train_loss"],5),
                                round(net1_dropout.train_history_[-1]["valid_loss"],5),
                                round(net1_dropout.train_history_[-1]["dur"],5)] )
    
    subfig = fig.add_subplot(2, 5, index+1, xticks=[], yticks=[])
    plt.plot(train_loss, linewidth=3, color="blue", linestyle="dashed", label="train")
    plt.plot(valid_loss, linewidth=3, color="blue", linestyle="solid", label="valid")
    plt.grid()
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.ylim(5*1e-4, 1e-2)
    plt.xlim(0,1000)
    plt.yscale("log")
    subfig.set_title('Dropout Rate: ' + str(num))

plt.show()
performance_table_3  

In [ ]:
net1_dropout = NeuralNet(layers=[('input', layers.InputLayer), # three layers
                                 ('hidden', layers.DenseLayer),
                                 ('dropout1', layers.DropoutLayer),
                                 ('output', layers.DenseLayer),],
                         
                         input_shape=(None, 9216),  # 96x96 input pixels per batch
                         hidden_num_units=500,  # number of units in hidden layer
                         output_nonlinearity=None,  # output layer uses identity function
                         output_num_units=30,  # 30 target values
                         dropout1_p=0.1,
                 
                         update=nesterov_momentum, # optimization parameters
                         update_learning_rate=0.01,
                         update_momentum=0.9,
                         
                         regression=True,  # flag to indicate we're dealing with regression problem
                         max_epochs=1000,  # we want to train this many epochs
                         verbose=1,)


net1_dropout.fit(X, y)



In [ ]:
import theano.tensor as T
#from lasagne.nonlinearities import softmax
#from lasagne.layers import InputLayer, DenseLayer
from lasagne import regularization
from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.regularization import regularize_layer_params

net1_l2norm = NeuralNet(layers=[('input', layers.InputLayer), # three layers
                                ('hidden', layers.DenseLayer),
                                ('output', layers.DenseLayer),],
                         
                        input_shape=(None, 9216),  # 96x96 input pixels per batch
                        hidden_num_units=500,  # number of units in hidden layer
                        output_nonlinearity=None,  # output layer uses identity function
                        output_num_units=30,  # 30 target values
                          
                        loss = lasagne.objectives.mse + 0.0001*lasagne.regularization.l2(output),
                        
                        update=nesterov_momentum, # optimization parameters
                        update_learning_rate=0.01,
                        update_momentum=0.9,
                         
                        regression=True,  # flag to indicate we're dealing with regression problem
                        max_epochs=1000,  # we want to train this many epochs
                        verbose=1,)


net1_l2norm.fit(X, y)


In [ ]:
from nolearn.lasagne import visualize